In [ ]:
import os
import math
import glob
import random
import itertools
import numpy as np
from PIL import Image
from skimage import io

import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F

import torch.utils.data
from torchvision.utils import save_image
import torchvision
from torch.autograd import Variable
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class MyDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dir_A, dir_B, transform=None):
        """
        Args:

        """
        self.dir_A = glob.glob(os.path.join(dir_A, '*.jpg'))
        self.dir_B = glob.glob(os.path.join(dir_B, '*.jpg'))
        self.transform = transform

    def __len__(self):
        return min(len(self.dir_A), len(self.dir_B))

    def __getitem__(self, idx):

        image_A = Image.open(self.dir_A[idx])

        image_B = Image.open(self.dir_B[idx])
        
        if self.transform:
            image_A = self.transform(image_A)
            image_B = self.transform(image_B)

        return image_A, image_B
    
class DatasetFromFolder(Dataset):
    def __init__(self, photo_path, sketch_path, transform = None):
        super(DatasetFromFolder, self).__init__()
        
        self.photo_path = photo_path
        self.sketch_path = sketch_path

        self.image_filenames = [x for x in os.listdir(self.photo_path) if 
                               any(x.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])]

        transform_list = [transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]

        self.transform = transforms.Compose(transform_list)
    
    
    def load_img(self, filepath):
        img = Image.open(filepath).convert('RGB')
        #img = img.resize((256, 256), Image.BICUBIC)
        return img
    
    def __getitem__(self, index):
        # Load Image}  
   
        input = self.load_img(os.path.join(self.photo_path, self.image_filenames[index]))
        input = self.transform(input)
        target = self.load_img(os.path.join(self.sketch_path, self.image_filenames[index]))
        target = self.transform(target)

        return input, target

    def __len__(self):
        return len(self.image_filenames)

### Generator

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        #Downsampling layers
        
        #c7s1-32
       
        self.conv_1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False) 
        
        #d64
        self.conv_2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_2 = nn.BatchNorm2d(128, affine=True, track_running_stats=True)
        
        #d128
        self.conv_3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_3 = nn.BatchNorm2d(256, affine=True, track_running_stats=True)
        
        self.conv_4 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_4 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_5 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_5 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_6 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_6 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_7 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_7 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_8 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_8 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
           
        #UpSampling
        self.conv_9 = nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_9 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_10 = nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_10 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_11 = nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_11 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_12 = nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_12 = nn.BatchNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_13 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_13 = nn.BatchNorm2d(256, affine=True, track_running_stats=True)
        
        self.conv_14 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_14 = nn.BatchNorm2d(128, affine=True, track_running_stats=True)
          
        #u32
        self.conv_15 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2,
                                         padding=1,
                                         bias=True)
        self.norm_15 = nn.BatchNorm2d(64, affine=True, track_running_stats=True)
        
        #output layer
        
        self.conv_out = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=True)
        
    def forward(self, x):
        '''
        Inputs:
            x: (batch x 3 x 256 x 256)
        Outputs:
            image: (batch x 3 x 256 x 256)
        '''
    
        x = self.conv_1(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_2(x)
        x = self.norm_2(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_3(x)
        x = self.norm_3(x)
        x = F.leaky_relu(x, 0.2)

        x = self.conv_4(x)
        x = self.norm_4(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_5(x)
        x = self.norm_5(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_6(x)
        x = self.norm_6(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_7(x)
        x = self.norm_7(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_8(x)
        x = self.norm_8(x)
        x = F.leaky_relu(x, 0.2)
        
        #upsampling
        x = self.conv_9(x)
        x = self.norm_9(x)
        x = F.relu(x)
        
        x = self.conv_10(x)
        x = self.norm_10(x)
        x = F.relu(x)
        
        x = self.conv_11(x)
        x = self.norm_11(x)
        x = F.relu(x)
        
        x = self.conv_12(x)
        x = self.norm_12(x)
        x = F.relu(x)
        
        x = self.conv_13(x)
        x = self.norm_13(x)
        x = F.relu(x)
        
        x = self.conv_14(x)
        x = self.norm_14(x)
        x = F.relu(x)
        
        x = self.conv_15(x)
        x = self.norm_15(x)
        x = F.relu(x)

        
        x = self.conv_out(x)
        x = torch.tanh(x)
        
        return x

### Discriminator

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv_1 = nn.Conv2d(6, 64, kernel_size=4, stride=2, padding=1, bias=True)
        
        self.conv_2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_2 = nn.InstanceNorm2d(128, affine=True, track_running_stats=True)
        
        self.conv_3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=True)
        self.norm_3 = nn.InstanceNorm2d(256, affine=True, track_running_stats=True)
        
        self.conv_4 = nn.Conv2d(256, 512, kernel_size=4, padding=1, bias=True)
        self.norm_4 = nn.InstanceNorm2d(512, affine=True, track_running_stats=True)
        
        self.conv_output = nn.Conv2d(512, 1, kernel_size=4, padding=1, bias = False)
       
    def forward(self, x):
        '''
        Inputs:
            x: (batch, 3, 128, 128)
        Output:
            out: (batch, 1)
        '''
        
        x = self.conv_1(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_2(x)
        x = self.norm_2(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_3(x)
        x = self.norm_3(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_4(x)
        x = self.norm_4(x)
        x = F.leaky_relu(x, 0.2)
        
        x = self.conv_output(x)
        x = torch.sigmoid(x)
        
        return x

### Loss

In [6]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [7]:
criterion_GAN = torch.nn.MSELoss()
criterion_l1 = torch.nn.L1Loss()

In [8]:
optimizer_gen = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_disc = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

In [9]:
batch_size = 4
dataset = DatasetFromFolder('facades/train/a', 'facades/train/b')
dataloader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=True, num_workers=4)

Tensor = torch.cuda.FloatTensor
input_A = Tensor(batch_size, 3, 256, 256)
input_B = Tensor(batch_size, 3, 256, 256)

In [14]:
epoch = 0

### Input A is image of summer
### Input B is image of winter

In [ ]:

while epoch<200:
    for en, x in enumerate(dataloader):
#         x = x.to(device)
        
        input_A = Tensor(x[0].size(0), 3, 256, 256)
        input_B = Tensor(x[0].size(0), 3, 256, 256)
        real_A = Variable(input_A.copy_(x[0])) 
        real_B = Variable(input_B.copy_(x[1]))
        
            
        target_real = Variable(Tensor(real_A.size(0), 1, 30, 30).fill_(1.0), requires_grad=False)
        target_fake = Variable(Tensor(real_B.size(0), 1, 30, 30).fill_(0.0), requires_grad=False)
        
        ### Disctiminator LOSS ###     
        optimizer_disc.zero_grad()
        
        fake_B = generator(real_A)    
        disc_fake = discriminator(torch.cat((real_A, fake_B.detach()), 1))
        fake_loss = criterion_GAN(disc_fake, target_fake)
        
        disc_real = discriminator(torch.cat((real_B, real_A), 1))
        real_loss = criterion_GAN(disc_real, target_real)
        
        disc_loss = (fake_loss + real_loss) * 0.5        
        disc_loss.backward()
        optimizer_disc.step()
        

        
        ### Generator Loss ###
        optimizer_gen.zero_grad()
        
        disc_out = discriminator(torch.cat((fake_B, real_A), 1))
        gen_loss = criterion_GAN(disc_out, target_real) 
        
          # L1 loss #
        loss_l1 = criterion_l1(fake_B, real_B)
        
        all_gen_loss = loss_l1 + gen_loss
        all_gen_loss.backward()
        optimizer_gen.step()
    
        if en%100==0:          
            img = torchvision.utils.make_grid(torch.cat([real_A, fake_B], dim=0), nrow=4)
            save_image(img, filename=('pix2pix_outputs/'+str(epoch)+'_'+str(en)+'.jpg'))
            
    print("The generator loss {0:2f}, discriminator Loss {1:2f}".format(disc_loss.item(), all_gen_loss.item()))
    epoch += 1
    

The generator loss 0.000000, discriminator Loss 0.111354
The generator loss 0.000000, discriminator Loss 0.113223
The generator loss 0.000000, discriminator Loss 0.119994
The generator loss 0.000000, discriminator Loss 0.153607
The generator loss 0.000000, discriminator Loss 0.097045
The generator loss 0.000000, discriminator Loss 0.137227
The generator loss 0.000000, discriminator Loss 0.176088
The generator loss 0.000000, discriminator Loss 0.140484
The generator loss 0.000000, discriminator Loss 0.131814
The generator loss 0.000000, discriminator Loss 0.108552
The generator loss 0.000000, discriminator Loss 0.121932
The generator loss 0.000000, discriminator Loss 0.146454
The generator loss 0.000000, discriminator Loss 0.121909
The generator loss 0.000000, discriminator Loss 0.119357
The generator loss 0.000000, discriminator Loss 0.118586
The generator loss 0.000000, discriminator Loss 0.123267
The generator loss 0.000000, discriminator Loss 0.104813
The generator loss 0.000000, di

In [12]:
test_dataset = MyDataset('data/testA', 'data/testB', transform=transforms.Compose([
                                               transforms.Resize(128),
                                               transforms.ToTensor()]))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=4)


for en, test in enumerate(dataloader):
    generator_A2B.eval()
    real_A = Variable(test[0]).to(device)
    fake_B = generator_A2B(real_A)
    img = torchvision.utils.make_grid([real_A.squeeze(0), fake_B.squeeze(0)], nrow=2)
    save_image(img, filename=('test_outputs/'+'_'+str(en)+'.jpg'))

NameError: name 'generator_A2B' is not defined